In [10]:
#hide
#default_exp tools.print_info
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Print info

> Prints information about the experiment indicated as argument: 
- path to experiment data
- parameters
- name of evaluation scores in experiment csv data
- name of metrics monitored during training
- scores obtained across all the runs 

In [11]:
#export
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from IPython.display import display
import argparse

import sys
sys.path.append('.')
sys.path.append('src')

from hpsearch.config.hpconfig import get_path_experiments, get_path_results, get_experiment_manager
import hpsearch.utils.experiment_utils as ut
from hpsearch.tools.metric_visualization import include_best_and_last_experiment
import hpsearch.config.hp_defaults as dflt

In [12]:
#for tests
import pytest
from hpsearch.examples.dummy_experiment_manager import generate_data

## print_info

In [13]:
#export
def print_info (experiments=[-1], base=None, folder=None, display_all=False, include_best=False, 
                op=None, metric=None, round_digits=2, compare=True, compact=0, run_number=0, 
                manager_path=dflt.manager_path):
        
    em = get_experiment_manager (manager_path=manager_path)
    if path_experiments is not None or folder is not None:
        em.set_path_experiments (path_experiments=path_experiments, folder=folder)
    if metric is not None:
        em.key_score = metric
    if op is not None:
        em.op = op
        
    path_experiments = em.path_experiments
        
    df = pd.read_pickle (path_experiments/'experiments_data.pk')

    metric_column = f'{run_number}_{em.key_score}'
        
    experiments = include_best_and_last_experiment ([em.key_score], experiments=experiments, 
                                                    run_number=run_number, op=self.op)


    df_scores = None
    print ('\n*****************************')
    for experiment in experiments:
        parameters = ut.get_parameters_columns(df.loc[[experiment]], True)
        print (f'\nparameters for {experiment}:')
        df2 = df.copy()
        df2[metric] = df.loc[experiment, metric_column]
        display (df2.loc[experiment, parameters + [metric]])
        print ('scores for all experiments:')
        df_scores = ut.get_experiment_scores(df.loc[[experiment]], suffix_results='_%s' %metric, remove_suffix=True)
        display(df_scores.round(round_digits))

        path_results = em.get_path_results (experiment, run_number)
        print (f'path to results: {path_results}')
        scores_names = ut.get_scores_names (df, experiment=experiment, run_number=run_number)
        print (f'scores names: {scores_names}')
        monitored_metrics = ut.get_monitored_training_metrics (experiment, run_number, path_results=path_results)
        print (f'monitored metrics: {monitored_metrics}')
        print ('\n*****************************')

### usage

In [14]:
#export tests.tools.test_print_info
def test_print_info ():
    em = generate_data ('print_info')
    
    print_info (manager_path=em.manager_path)
    
    em.remove_previous_experiments ()

In [15]:
tst.run (test_print_info, tag='dummy')

running test_print_info

*****************************

parameters for 8:


epochs                       30
noise                       0.1
offset                      0.6
rate                       0.03
validation_accuracy    0.945403
Name: 8, dtype: object

scores for all experiments:


,0,1,2,3,4
8,0.95,0.91,0.9,0.89,0.8


path to results: test_print_info//experiments/00008/0
scores names: ['test_accuracy', 'validation_accuracy']
monitored metrics: ['validation_accuracy', 'test_accuracy', 'accuracy']

*****************************


## parse_args

In [16]:
#export
def parse_args(args):
    parser = argparse.ArgumentParser(description='print table') 
    # Datasets
    parser.add_argument('--folder', type=str, default=None, help='name of experiments folder')
    parser.add_argument('--path_experiments', type=str, default=None, help='full experiments path')
    parser.add_argument('-m', '--metric', type=str, default=None, help='metric score')
    parser.add_argument('-e', type=int, nargs='+', default=[-1, -2], help='experiment numbers')
    parser.add_argument('-a', type=bool, default=False)
    parser.add_argument ('-b', '--best', action= "store_true", help='include experiment with best performance (on given run id!!)')
    parser.add_argument ('-r', '--run', type=int, default=0, help='run id') 
    parser.add_argument('-n', '--no_comp', action= "store_true", help='do not perform comparison')
    parser.add_argument('--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--op', default=None, type=str)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('-p', '--path', default=dflt.manager_path, type=str)
    pars = parser.parse_args(args)
    
    return pars

def parse_arguments_and_run (args):
    
    pars = parse_args(args)

    print_info (experiments=pars.e, path_experiments = pars.path_experiments, folder=pars.folder, 
                display_all=pars.a, include_best=pars.best, op=pars.op, metric=pars.metric, 
                round_digits=pars.round, compare=not pars.no_comp, compact=pars.compact, 
                run_number=pars.run, manager_path=pars.path)

def main():
    parse_arguments_and_run (sys.argv[1:])

### usage

In [17]:
#export tests.tools.test_print_info
def test_parse_args ():
    em = generate_data ('parse_args')
    
    args = ['-e', '4', '3',
       '--compact', '3',
       '--round', '3',
       '-p', em.manager_path]
    parse_arguments_and_run (args)
    
    em.remove_previous_experiments ()

In [18]:
tst.run (test_parse_args, tag='dummy')

running test_parse_args

*****************************

parameters for 4:


epochs                       15
noise                       0.1
offset                      0.3
rate                       0.03
validation_accuracy    0.815981
Name: 4, dtype: object

scores for all experiments:


,0,1,2,3,4
4,0.816,0.656,0.8,1.0,0.819


path to results: test_parse_args//experiments/00004/0
scores names: ['test_accuracy', 'validation_accuracy']
monitored metrics: ['validation_accuracy', 'test_accuracy', 'accuracy']

*****************************

parameters for 3:


epochs                      15
noise                      0.1
offset                     0.1
rate                      0.03
validation_accuracy    0.54907
Name: 3, dtype: object

scores for all experiments:


,0,1,2,3,4
3,0.549,0.604,0.471,0.492,0.491


path to results: test_parse_args//experiments/00003/0
scores names: ['test_accuracy', 'validation_accuracy']
monitored metrics: ['validation_accuracy', 'test_accuracy', 'accuracy']

*****************************
